In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading data:

- Source: https://www.kaggle.com/c/mercari-price-suggestion-challenge/data

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7559/44327/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1664722407&Signature=o9jXoABzgz8zmCfUJx%2B6SNds8QcCmsEqGm2ZBrO1X3dh%2FfCnyyT5eRwgec3evuoIkAUGOY36pujSk2IDq3XdHJV9qxffZEmdmgoobpm6cn0wx3PxpUT4Yam6rZaT6R4oo7Vkqey%2B8x0o%2FbZydJsTuYOJXp7OiieJxxQo0WLLBjt4aBHs4DOz0r6HnW6oIHDqOvPsyW1QJjbFPomCV0BfZtk%2FDsc%2BxaR6Q6us9kSq93PVrsYrSslm1SXBQyWAJ%2BbTQCiOQDymlahDE%2FEtc9CmhiFyjbDnfEBUIFRBgIVwsb1WbLVZJOU4ce0L6t5PaK35ZOJ7bUy5lqunWhO3T85e3A%3D%3D&response-content-disposition=attachment%3B+filename%3Dmercari-price-suggestion-challenge.zip" -c -O 'mercari-price-suggestion-challenge.zip'
!unzip mercari-price-suggestion-challenge.zip
!7z x train.tsv.7z -o*
!7z x test.tsv.7z -o*
!7z x test_stg2.tsv.zip -o*

# deleting other un-necessary files on colab
import os
os.remove("/content/mercari-price-suggestion-challenge.zip")
os.remove("/content/sample_submission.csv.7z")
os.remove("/content/sample_submission_stg2.csv.zip")
os.remove("/content/test.tsv.7z")
os.remove("/content/train.tsv.7z")
os.remove("/content/test_stg2.tsv.zip")

import shutil
shutil.rmtree("sample_data")

--2022-09-29 14:54:03--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7559/44327/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1664722407&Signature=o9jXoABzgz8zmCfUJx%2B6SNds8QcCmsEqGm2ZBrO1X3dh%2FfCnyyT5eRwgec3evuoIkAUGOY36pujSk2IDq3XdHJV9qxffZEmdmgoobpm6cn0wx3PxpUT4Yam6rZaT6R4oo7Vkqey%2B8x0o%2FbZydJsTuYOJXp7OiieJxxQo0WLLBjt4aBHs4DOz0r6HnW6oIHDqOvPsyW1QJjbFPomCV0BfZtk%2FDsc%2BxaR6Q6us9kSq93PVrsYrSslm1SXBQyWAJ%2BbTQCiOQDymlahDE%2FEtc9CmhiFyjbDnfEBUIFRBgIVwsb1WbLVZJOU4ce0L6t5PaK35ZOJ7bUy5lqunWhO3T85e3A%3D%3D&response-content-disposition=attachment%3B+filename%3Dmercari-price-suggestion-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.123.128, 142.250.98.128, 142.251.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.123.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 422739618 (403M) [application/zip]
Saving to: ‘mercari-price-su

# Imports:

In [ ]:
!pip install category_encoders
!pip install scikit-optimize
!pip install -q -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 4.1 MB/s 
     |████████████████████████████████| 135 kB 5.3 MB/s 
     |████████████████████████████████| 1.6 MB 50.9 MB/s 


In [ ]:
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import ( MinMaxScaler, StandardScaler, Normalizer, 
                                    OneHotEncoder )
from sklearn.linear_model import SGDRegressor

from xgboost import XGBRegressor

from scipy.sparse import hstack
from scipy import sparse

from functools import partial
from skopt.plots import plot_convergence
from skopt import gp_minimize
from skopt import space

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input
import keras_tuner as kt

from prettytable import PrettyTable
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

import re

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Final prediction pipeline:

**Loading all the necessary objects:**

In [ ]:
final_features = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/final_features.joblib")
brand_counts = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/brand_counts.joblib")
grp_by_stats = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/grp_by_stats.joblib")
group_by_combos = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/group_by_combos.joblib")

# Featurizers
target_encoder = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/target_encoder.joblib")
one_hot_encoder = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/one_hot_encoder.joblib")
ctv_text = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/ctv_text.joblib")
tfv_text = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/tfv_text.joblib")
ctv_name_and_brand = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/ctv_name_and_brand.joblib")
tfv_name_and_brand = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/tfv_name_and_brand.joblib")

# feature scalers
min_max_scaler_1 = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/min_max_scaler_1.joblib")
min_max_scaler_2 = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/featurizers/min_max_scaler_2.joblib")

# models
mlp_ds1 = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/models/mlp_ds1")
mlp_ds2 = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/models/mlp_ds2")
meta_lr = joblib.load("/content/drive/MyDrive/Colab Notebooks/AppliedAI/Assignments/CS-1-MERCARI/models/meta_lr.pkl")

## Preprocessing of the data:

**Helper functions:**

In [ ]:
def split_cat(category_name):
    """
    Split Category into 3 different levels.
    """
    categories = ["no_main_cat", "no_subcat_1", "no_subcat_2"]
    if isinstance(category_name, str):
        split_values = category_name.split("/")
        for i, cat in enumerate(split_values):
            if cat and (i < 3):
                categories[i] = cat
    return categories

def preprocess_brand_name(txt):
    """
    Preprocess brand name text.
    """
    # remove html tags
    sent = re.sub(r"http\S+", "", txt)
    sent = BeautifulSoup(sent, 'lxml').get_text()
    # clean up certain special symbols in text
    sent = sent.replace("′", "'").replace("’", "'")
    sent = re.sub('[^A-Za-z0-9_\-]+', ' ', sent)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\n', ' ')
    # lower the text
    sent = str(sent).lower()
    # removing any extra white spaces
    sent = ' '.join(word for word in sent.split())
    sent = sent.strip()
    return sent

def decontracted(phrase):
    """
    decontract shorthand words used in the data.
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"cannot", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# removing the following words from the stop words list: 'no', 'nor', 'not'
STOP_WORDS = set(stopwords.words("english"))
STOP_WORDS = STOP_WORDS - {'no', 'nor', 'not'}
# let's add "br" because after removing html tags such as "<br />" using 
# Beatiful Soup we may still be left with "br".
# instead of <br /> if we had <br/> these tags would have been removed with 
# Beatiful Soup.
STOP_WORDS.update("br")

# initialize stemmer
stemmer = SnowballStemmer("english")

# let's keep ? and ! marks in the item name and desc without removing them, 
# since it could mean seller was trying to convey something in a different way 
# to grab the buyer's attention and it could have had a first impression on the 
# buyer when the name or desc was read. 
def preprocess_item_name_and_desc(txt, remove_stopwords=True):
    """
    Preprocess item_name and item_description.
    """
    # remove html tags
    sent = re.sub(r"http\S+", "", txt)
    sent = BeautifulSoup(sent, 'lxml').get_text()
    # lower the text and clean up certain special symbols in text
    sent = sent.replace("′", "'").replace("’", "'")
    sent = str(sent).lower()
    sent = decontracted(sent)
    sent = re.sub('[^A-Za-z0-9_\!\?]+', ' ', sent)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\n', ' ')
    sent = sent.replace('\\"', ' ')
    # remove stopwords if required and remove any extra white spaces.
    if remove_stopwords:
        sent = ' '.join(
                   stemmer.stem(e) for e in sent.split() if e not in STOP_WORDS)
    else:
        sent = " ".join(stemmer.stem(e) for e in sent.split())
    sent = sent.strip()
    return sent

def tokenize(txt):
    """
    Tokenize text using nltk word_tokenize and sent_tokenize.
    """
    tokens = []
    for s in sent_tokenize(txt):
        tokens.extend(word_tokenize(s))
    return tokens

**Function to preprocess the data:**

In [ ]:
def preprocess_data(df):
    """
    Preprocess input data as necessary by the models for prediction.

    Parameters
    ----------
    df: pandas.DataFrame
        Input product data whose price prediction is to be made. 
        This can be single point or a set of points. 
    
    Returns
    -------
    preprocessed_df: pandas.DataFrame
        Dataframe with preprocessed data.
    """
    # Fill empty string("") for Null Values:
    df.fillna('', inplace=True)
    df.loc[df["item_description"].str.lower() == 
            "no description yet", "item_description"] = ""
    # ==========================================================================

    # Creating indicators "missing_brand_name" and "missing_item_desc":
    df["missing_brand_name"] = (df["brand_name"]=="").map(int)
    df["missing_item_desc"] = (df["item_description"]=="").map(int)
    # ==========================================================================

    # Split Category:
    df["main_cat"], df["subcat_1"], df["subcat_2"] = zip(
                                        *df["category_name"].apply(split_cat))
    # drop the original "category_name" column as it's not used anymore.
    df.drop("category_name", inplace=True, axis=1)
    # ==========================================================================

    # Clean up text columns - "brand_name", "item_name" and "item_description":=
    df["preproc_brand_name"] = df["brand_name"].apply(preprocess_brand_name)
    # drop the original brand_name data as it's not used anymore.
    df.drop("brand_name", axis=1, inplace=True)
    # preprocess item name without removing stop words
    df["preproc_name"] = df["name"].apply(preprocess_item_name_and_desc,
                                                    remove_stopwords=False)
    # preprocess item name by removing stop words
    df["preproc_desc"] = df["item_description"].apply(
                                                preprocess_item_name_and_desc)
    # ==========================================================================

    # Combining "item_name", "brand_name" and "item_description" into "text" 
    # column and combining "item_name" and "brand_name" into "name_and_brand" 
    # column:
    df["preproc_text"] = (df["preproc_name"]+" "+df["preproc_desc"]+" "+
                            df["preproc_brand_name"])
    df["preproc_name_and_brand"] = (df["preproc_name"]+" "+
                                    df["preproc_brand_name"])
    # replace multiple whitespaces with a single space
    df['preproc_text'] = df['preproc_text'].replace(r'\s+', ' ', regex=True)
    df['preproc_name_and_brand'] = df['preproc_name_and_brand'].replace(
        r'\s+', ' ', regex=True)
    # ==========================================================================

    # Calculating the length and word count for the below:
    # item_name:
    # "name_length", "preproc_name_length", "name_word_count".
    # item_description:
    # "log_desc_length". "log_preproc_desc_word_count".

    # "item_name"
    df["name_length"] = df["name"].apply(len)
    df["preproc_name_length"] = df["preproc_name"].apply(len)
    df["name_word_count"] = df["name"].apply(lambda x: len(word_tokenize(x)))
    # "item_description"
    df["log_desc_length"] = df["item_description"].apply(
        lambda x: np.log1p(len(x)))
    df["log_preproc_desc_word_count"] = df["preproc_desc"].apply(
        lambda x: np.log1p(len(tokenize(x))))
    # drop the columns that are not needed anymore "item_name", 
    # "item_description" and "preproc_desc".
    df.drop("name", axis=1, inplace=True)
    df.drop("item_description", axis=1, inplace=True)
    df.drop(["preproc_desc"], axis=1, inplace=True)
    df.drop(["preproc_name"], axis=1, inplace=True)
    # ==========================================================================

    # Assign the brand count values calculated from train data for test data:
    df["brand_count"] = df["preproc_brand_name"].apply(
                                     lambda x: brand_counts.get(x, 0))
    # ==========================================================================

    # Assign group-by statistics obtained from train data to test data:
    # group_by stats for cv data
    for grp_name, grp_combo in group_by_combos.items():
        df = df.merge(grp_by_stats[grp_name], how='left', on=grp_combo)
    # fill 0 to any nan values that may have resulted when a particular combo 
    # value present in the main dataframe is not found in any of the dataframes 
    # containing group_by stats.
    df.fillna(0, inplace=True)
    # this is not necessary for train set but is needed for test set as it may 
    # contain unseen values for a group_by combo.
    # dropping "preproc_brand_name", since its already merged into "preproc_text"
    df.drop("preproc_brand_name", axis=1, inplace=True)
    # ==========================================================================
    
    cols = set()
    for key, vals in final_features.items():
        cols.update(set(vals))
    assert cols == (set(df.columns) - {"test_id", "train_id", "bins", "price"})
    return df

**Note:** It is found that the "categories" column in test data has 4 levels whereas the train data had only 3 levels. Hence modifying the function to incorporate only 3 levels out of the 4. 

## Featurization of the data:


1. **Dataset Set 1:**
One-Hot encoded categories and shipping_id, Ordinal encoded item_condition_id + TFIDF encoded text with bigrams and other numerical features obtained from feature engineering.
2. **Dataset Set 2:**
Target encoded categories and shipping_id, Ordinal encoded item_condition_id + BOW encoded text with bigrams and other numerical features obtained from feature engineering.


In [ ]:
def create_dataset_1(row_df):
    """
    Return features pertaining to Dataset-1(One-Hot encoded categories and 
    shipping_id, Ordinal encoded item_condition_id + TFIDF encoded text with 
    bigrams and other numerical features obtained from feature engineering).

    Parameters:
    -----------
    row_df: pandas.DataFrame
        Input product data for which features pertaining to Dataset-1 is to be 
        returned. 

    Returns:
    --------
    features: scipy.sparse.csr.csr_matrix
        Features pertaining to dataset-1
    """
    # Seperating Features:
    
    # indicators
    ind_cols_te = row_df.loc[:, final_features['ind_cols']].astype(int)
    # Categories and Shipping ID
    categories_shipping_te = row_df.loc[:, final_features['cats_and_shipping']]
    # Preproc name and brand
    preproc_name_and_brand_te = row_df.loc[:, ['preproc_name_and_brand']]
    # Preproc text
    preproc_text_te = row_df.loc[:, ['preproc_text']]
    # Other Engineered features:
    count_feats_te = row_df.loc[:, final_features['count_feats']]
    # group_by features
    group_by_feat_te = row_df.loc[:, final_features['grp_by_features']]

    # TFIDF:
    tfidf_text_te = tfv_text.transform(preproc_text_te['preproc_text'])
    tfidf_name_and_brand_te = tfv_name_and_brand.transform(
        preproc_name_and_brand_te['preproc_name_and_brand'])

    # One-Hot Encoding Categories and Shipping Id:
    one_hot_encoder.handle_unknown = "ignore"
    enc_categories_shipping_te = one_hot_encoder.transform(categories_shipping_te)

    # scale test data
    scaled_features_te = min_max_scaler_1.transform(
        np.hstack((ind_cols_te.values, count_feats_te.values, 
                group_by_feat_te.values))
    )

    features = hstack((scaled_features_te, enc_categories_shipping_te, 
                       tfidf_name_and_brand_te, tfidf_text_te), format="csr")
    assert(features.shape[1] == 171883)
    
    return features


def create_dataset_2(row_df):
    """
    Return features pertaining to Dataset-2(Target encoded categories and 
    shipping_id, Ordinal encoded item_condition_id + BOW encoded 
    text with bigrams and other numerical features obtained from feature 
    engineering).

    Parameters:
    -----------
    row_df: pandas.DataFrame
        Input product data for which features pertaining to Dataset-2 is to be 
        returned. 

    Returns:
    --------
    features: scipy.sparse.csr.csr_matrix
        Features pertaining to dataset-2
    """
    # Seperating Features:
    
    # indicators
    ind_cols_te = row_df.loc[:, final_features['ind_cols']].astype(int)
    # Categories and Shipping ID
    categories_shipping_te = row_df.loc[:, final_features['cats_and_shipping']]
    # Preproc name and brand
    preproc_name_and_brand_te = row_df.loc[:, ['preproc_name_and_brand']]
    # Preproc text
    preproc_text_te = row_df.loc[:, ['preproc_text']]
    # Other Engineered features:
    count_feats_te = row_df.loc[:, final_features['count_feats']]
    # group_by features
    group_by_feat_te = row_df.loc[:, final_features['grp_by_features']]

    # BOW:
    bin_bow_text_te = ctv_text.transform(
        preproc_text_te['preproc_text'])
    bin_bow_name_and_brand_te = ctv_name_and_brand.transform(
        preproc_name_and_brand_te['preproc_name_and_brand'])

    # Target Encoding Categories and Shipping Id:
    enc_categories_shipping_te = target_encoder.transform(
        categories_shipping_te)

    # scale test data
    scaled_features_te = min_max_scaler_2.transform(
        np.hstack((ind_cols_te.values, count_feats_te.values, 
                enc_categories_shipping_te.values, group_by_feat_te.values))
    )

    features = hstack(
        (scaled_features_te, bin_bow_name_and_brand_te, bin_bow_text_te), 
                      format="csr")
    assert(features.shape[1] == 170890)
    return features

**Note:** New categories were found in the test data which made our assumption(that categories could be set of predefined values in the platform) wrong. 

Hence `one_hot_encoder.handle_unknown = "ignore"` is used above to ignore the unknown categories.

We could also follow an approach of moving all the infrequent categories into a seperate category called 'infrequent'.

## Prediction:

In [ ]:
def final_predict_prices(X):
    """
    Predict price of the input product data.
    
    Parameters:
    -----------
    X: pandas.DataFrame
        Input product data whose price prediction is to be made. 
        This can be single point or a set of points. 
    
    Returns:
    --------
    Pred_df: pandas.DataFrame
        predicted prices for the product data.
        If the input data had "test_id" column, then it will be used as the index.
    """
    test_id_in_cols = "test_id" in X.columns
    pred_df_index = X["test_id"] if test_id_in_cols else range(len(X))
    pred_df = pd.DataFrame(columns=["price"], index=pred_df_index)
    
    # iterate through each row and generate predictions
    for i in tqdm(range(len(X))):
        row_df = X.iloc[i: i+1]
        # preprocess data
        row_df = preprocess_data(row_df)
        # create datasets with different featurizations
        features_1 = create_dataset_1(row_df)
        features_2 = create_dataset_2(row_df)
        # make prediction
        pred_1 = mlp_ds1.predict(features_1)
        pred_2 = mlp_ds2.predict(features_2)
        final_pred = np.abs(np.expm1(meta_lr.predict(
            np.hstack((pred_1, pred_2)))))[0]
        # store prediction
        if test_id_in_cols:
            pred_df.loc[row_df["test_id"], "price"] = final_pred
        else:
            pred_df.loc[i, "price"] = final_pred
            
    return pred_df

## RMSLE:

In [ ]:
def final_evaluation_metric(X, y):
    """
    Returns the RMSLE of the predictions made by the best chosen models.

    Parameters
    ----------
    X: pandas.DataFrame
        Input product data whose price prediction is to be made. 
        This can be single point or a set of points. 
    
    y: pandas.Series
        Groundth truth data for the input X.

    Returns:
    --------
    rmsle: float
        RMSLE value calculated for predictions made by the models chosen.
    """
    pred_df = final_predict_prices(X)
    rmsle = mean_squared_log_error(
        y.values, pred_df['price'].values, squared=False)
    return rmsle

# Example:

In [ ]:
df_train = pd.read_csv('train.tsv/train.tsv', sep='\t')
df_train.shape

(1482535, 8)

Prediction:

In [ ]:
df_train[:2]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...


In [ ]:
pred_df = final_predict_prices(df_train[:2])
pred_df

  0%|          | 0/2 [00:00<?, ?it/s]

,price
0,7.826183
1,46.26404


In [ ]:
data = df_train[1:2]  # must be a df

# =====================================================
print("name: ", data["name"].values)
print("item_condition_id: ", data["item_condition_id"].values)
print("category_name: ", data["category_name"].values)
print("brand_name: ", data["brand_name"].values)
print("shipping: ", data["shipping"].values)
print("item_description: ", data["item_description"].values)

# =====================================================
pred_df = final_predict_prices(data)
print("\n Predicted Price: ", pred_df["price"].values)

name:  ['Razer BlackWidow Chroma Keyboard']
item_condition_id:  [3]
category_name:  ['Electronics/Computers & Tablets/Components & Parts']
brand_name:  ['Razer']
shipping:  [0]
item_description:  ['This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.']


  0%|          | 0/1 [00:00<?, ?it/s]


 Predicted Price:  [46.26403964046611]


RMSLE:

In [ ]:
df_train[:2]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,,10.0,1,
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...


In [ ]:
rmsle = final_evaluation_metric(df_train[:2], df_train[:2]["price"])
print("RMSLE: ", rmsle)

  0%|          | 0/2 [00:00<?, ?it/s]

RMSLE:  0.17549345997461738


In [ ]:
data = df_train[1:2]  # must be a df

# =====================================================
print("name: ", data["name"].values)
print("item_condition_id: ", data["item_condition_id"].values)
print("category_name: ", data["category_name"].values)
print("brand_name: ", data["brand_name"].values)
print("shipping: ", data["shipping"].values)
print("item_description: ", data["item_description"].values)

# =====================================================
rmsle = final_evaluation_metric(data, data["price"])
print("\nRMSLE: ", rmsle)

name:  ['Razer BlackWidow Chroma Keyboard']
item_condition_id:  [3]
category_name:  ['Electronics/Computers & Tablets/Components & Parts']
brand_name:  ['Razer']
shipping:  [0]
item_description:  ['This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.']


  0%|          | 0/1 [00:00<?, ?it/s]


RMSLE:  0.11454216850034538
